In [2]:
import os
import sys
import pickle
import random
import numpy as np
import pandas as pd
sys.path.append('../src/')
from configs import *
from training import *
from optimization import *
from training import seed_everything, load_dataset
from tqdm.notebook import tqdm

from matplotlib import rcParams
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
rcParams.update(fig_params)

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

## Load data and models

In [3]:
feature_list = ['physicochemical',
                'AAC',
                'CKSAAP type 1',
                'TPC type 1',
                'DPC type 1',
                'DDE',
                'GAAC',
                'CKSAAGP type 1',
                'GDPC type 1',
                'GTPC type 1',
                'Moran',
                'Geary',
                'NMBroto',
                'CTDC',
                'CTDT',
                'CTDD',
                'CTriad',
                'KSCTriad',
                'SOCNumber',
                'QSOrder',
                'PAAC',
                'APAAC',
                'ASDC',
                'AC',
                'CC',
                'ACC',
                'EAAC',
                'EGAAC',
                'AAIndex',
                'BLOSUM62',
                'ZScale'
               ]

comb, features = load_dataset(feature_list=feature_list, dataset_name='comb', features_folder=os.path.join('..', 'features'))
## Train dataset
train = comb[comb.Dataset=='train'].copy()
test  = comb[comb.Dataset=='test'].copy()

In [4]:
## Featurizer
featurizer = Featurizer(data_folder=os.path.join('..', 'data'))

## Model
with open(os.path.join('..', 'models', 'LightCPP_20.pickle'), 'rb') as handle:
    models = pickle.load(handle)
best_iteration = models['best_iteration']
n_models = 1

## Selected features
selected_features = ['CTDC_charge.G1',
                     'CTDD_hydrophobicity_ARGP820101.1.residue50',
                     'CKSAAP_IR.gap2',
                     'CTDD_normwaalsvolume.2.residue100',
                     'DDE_LA',
                     'ASDC_AS',
                     'PAAC_Xc1.V',
                     'APAAC_Pc2.Hydrophobicity.1',
                     'DDE_LP',
                     'CKSAAP_LL.gap4',
                     'APAAC_Pc1.A',
                     'CTDD_hydrophobicity_FASG890101.2.residue50',
                     'CKSAAGP_postivecharger.uncharger.gap1',
                     'KSCTriad_g5.g5.g5.gap1',
                     'ZScale_p1.z5',
                     'QSOrder_Grantham.Xr.T',
                     'ASDC_DM',
                     'CKSAAP_TR.gap4',
                     'PAAC_Xc1.G',
                     'CKSAAGP_uncharger.aromatic.gap9']

## Optimization loop

WARNING: This loop takes hours to execute.

In [6]:
seed_everything(SEED)
MAX_ITER = 50

peptide_sequences = test[test['CPP'] == 0][['ID','Sequence']].copy()
sequences = test[test['CPP'] == 0]['Sequence'].values
results = pd.DataFrame(index=pd.MultiIndex.from_product([sequences, range(1, MAX_ITER+1)],
                                                       names=['Input_peptide', 'Generation']),
    columns=['Initial_Penetration','Best_solution','Best_fitness', 'Penetration_score', 'Anomaly_score', 'Different_letters'])

for idx, row in peptide_sequences.iterrows():
    ID, peptide_sequence = row.ID, row.Sequence
    TARGET = peptide_sequence.strip()
    print(ID, TARGET)
    
    # Set-up
    len_gnome = len(TARGET)
    max_iter = MAX_ITER
    size_population = 500
    genes = '''ARNDCEQGHILKMFPSTWYV'''

    # Training features for anomaly detection
    feat_train = train[selected_features]
    clf_anomaly = LocalOutlierFactor(n_neighbors=20, novelty=True, contamination=0.1)
    clf_anomaly.fit(feat_train)

    # Sequence alignment settings
    aligner = Align.PairwiseAligner()
    aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
    aligner.target_internal_open_gap_score = -20
    aligner.query_internal_open_gap_score = -20

    generation = 1
    population = []

    optimization_info = {'Pop_size': size_population, 'max_iter': max_iter}

    params_fit = {'obj': my_fitness, 'target_ligand': TARGET, 'aligner': aligner, 'model': models['refit_model'], 'clf_anomaly': clf_anomaly, 
                 'featurizer': featurizer, 'best_iteration': best_iteration, 'n_models': n_models}

    features_TARGET = featurizer.compute_features(TARGET)
    print(f"Penetrability of target: {pred_penetration(features_TARGET, models['refit_model'], best_iteration, n_models)}")
    print(f"Anomaly of target: {anomaly_score(features_TARGET, clf_anomaly)}")

    # Create initial population
    max_diff_pct = 50  # set the percentage of different letters, user can change this value
    for _ in range(size_population):
        gnome = Individual.create_gnome(len_gnome, genes, TARGET, max_diff_pct)
        population.append(Individual(gnome, params_fit))

    # Define stopping criteria thresholds
    penetration_threshold = 0.95
    fitness_threshold = 0.1
    anomaly_threshold = 1.3

    best_sequences = {}

    while generation <= max_iter:
        # Sort the population in increasing order of fitness score
        population = sorted(population, key=lambda x: x.fitness)

        for individual in population:
            if individual.num_diff_residues not in best_sequences:
                best_sequences[individual.num_diff_residues] = individual
            else:
                if individual.fitness < best_sequences[individual.num_diff_residues].fitness:
                    best_sequences[individual.num_diff_residues] = individual

        # If the best solution in the population meets the stopping criteria, break the loop
        best_solution = population[0]
        features_best = featurizer.compute_features(''.join(best_solution.chromosome))
        penetration_score = pred_penetration(features_best, models['refit_model'], best_iteration, n_models)
        anomaly_score_value = anomaly_score(features_best, clf_anomaly)[0]

        # Including the condition for num_diff_residues
        if (penetration_score > penetration_threshold and anomaly_score_value < anomaly_threshold and best_solution.num_diff_residues <= 1):
            break
        # Including the condition for num_diff_residues
        if (best_solution.num_diff_residues == 0):
            break
        # Generate new offsprings for new generation
        new_generation = []

        # Perform Elitism, that mean 10% of fittest population
        # goes to the next generation
        s = int((10*size_population) / 100)
        new_generation.extend(population[:s])

        # From 50% of fittest population, Individuals
        # will mate to produce offspring
        s = int((90*size_population) / 100)
        for _ in range(s):
            parent1 = random.choice(population[:50])
            parent2 = random.choice(population[:50])
            child = parent1.mate(parent2, genes)
            new_generation.append(child)

        population = new_generation

        print(f"Generation: {generation} -- "
              f"Best solution: {''.join(population[0].chromosome)} -- "
              f"Best fitness: {population[0].fitness:.5f} -- "
              f"Prob_penetration: {penetration_score:.2f} -- "
              f"Anomaly score: {anomaly_score_value:.2f} -- "
              f"Different letters from target: {population[0].num_diff_residues}"
              )
        
        results.loc[(peptide_sequence, generation)]['Initial_Penetration'] = pred_penetration(features_TARGET, models['refit_model'], best_iteration, n_models)
        results.loc[(peptide_sequence, generation)]['Best_solution'] = ''.join(population[0].chromosome)
        results.loc[(peptide_sequence, generation)]['Best_fitness'] = population[0].fitness
        results.loc[(peptide_sequence, generation)]['Penetration_score'] = penetration_score
        results.loc[(peptide_sequence, generation)]['Anomaly_score'] = anomaly_score_value
        results.loc[(peptide_sequence, generation)]['Different_letters'] = population[0].num_diff_residues
        
        generation += 1

    # Print the best sequence for each number of different letters
    for num_diff_residues, individual in sorted(best_sequences.items()):
        print(f"Best solution with {num_diff_residues} different letters from target: "
              f"{''.join(individual.chromosome)} -- "
              f"Prob_penetration: {pred_penetration(featurizer.compute_features(''.join(individual.chromosome)), models['refit_model'], best_iteration, n_models):.2f} -- "
              f"Anomaly score: {anomaly_score(featurizer.compute_features(''.join(individual.chromosome)), clf_anomaly)[0]:.2f}"
              )
    
    results.to_pickle('../results/optimization_results.pickle')
    
    break

Negative_1 MILPTGPTSFKE
Penetrability of target: 0.36327373596143514
Anomaly of target: [1.09027296]
Generation: 1 -- Best solution: MILPRNPPSFKD -- Best fitness: 0.33333 -- Prob_penetration: 0.89 -- Anomaly score: 1.01 -- Different letters from target: 4
Generation: 2 -- Best solution: WILPTGPTSFKE -- Best fitness: 0.18108 -- Prob_penetration: 0.65 -- Anomaly score: 1.10 -- Different letters from target: 1
Generation: 3 -- Best solution: MILPRGPTAFKE -- Best fitness: 0.14470 -- Prob_penetration: 0.80 -- Anomaly score: 1.01 -- Different letters from target: 2
Generation: 4 -- Best solution: MILPRGPTSFKE -- Best fitness: 0.09524 -- Prob_penetration: 0.90 -- Anomaly score: 1.08 -- Different letters from target: 1
Generation: 5 -- Best solution: MILPRGPTSFKE -- Best fitness: 0.09524 -- Prob_penetration: 0.90 -- Anomaly score: 1.08 -- Different letters from target: 1
Generation: 6 -- Best solution: MILPRGPTSFKE -- Best fitness: 0.09524 -- Prob_penetration: 0.90 -- Anomaly score: 1.08 -- Di